# Pandas manual

<a name="contents"></a>
# Contents

- [Configuration](#conf)
- [Functions](#fnc)
- [Pandas](#pandas)
    - [simple df I/O](#simpleio)
    - [simple df handling](#handling)
    - [reshape df](#reshape)
    - [slice df](#slice)
    - [binning](#binning)
    - [row-wise operations](#rowwise)
        - [replace](#replace)
        - [map](#map)
        - [apply](#apply)
    - [aggregate operations](#aggregate)
        - [groupby](#groupby)
        - [aggregation](#aggr)
        - [transformation](#transf)
        - [filter](#filter)
    - [combining operations](#combine)
        - [concat](#concat)
        - [join](#join)
        - [merge](#merge)
    - [encoding](#encoding)
    - [complex operations](#complex)
- [Geopandas](#gpd)

---
<a name="conf"></a>
# Configuration

[Return to Contents](#contents)

In [ ]:
import os
import re
import csv
import sys
import pdb
import json
import pytz
import time
import string
import inspect
import zipfile
import unittest
import random as rnd
import datetime as dt
import multiprocessing
from operator import itemgetter
from itertools import groupby, chain

import numpy as np
import pandas as pd
from joblib import dump, load
from pympler import asizeof
from IPython.display import display, HTML, Image
from dateutil.relativedelta import relativedelta

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

display(HTML('<style>.container { width:90% !important; }</style>'))

---

<a name="fnc"></a>
# Functions

[Return to Contents](#contents)

---

<a name="pandas"></a>
# Pandas

[Return to Contents](#contents)

<a name="simpleio"></a>
### simple df I/O

[Return to Contents](#contents)

In [ ]:
# pandas series
series = pd.Series({'a': 1, 'b': 2, 'c': 3})

# series operations

#### create df

In [ ]:
# dataframe can be column-wise defined
df = pd.DataFrame({
    'index': ['user0', 'user1', 'user2', 'user3', 'user4', 'user5', 'user6', 'user7', 'user8', 'user9'],
    'name': ['Marco', 'John', 'Alejandro', 'Kylie', 'Abdul', 'Ingrid', 'Francois', 'Ilaria', 'Jesse', 'Tina'],
    'surname': ['Rossi', 'Doe', 'Lara', 'Liu', 'Said', 'Larsson', 'Degass', 'Rossi', 'Pinkman', 'Orwell'],
    'age': [32, 28, 41, 36, 39, 19, 22, 229, 31, 28],
    'email': ['m.rossi@gmail.com', 'j.doe@gmail.com', 'a.lala@gmail.com', np.NaN, 'a.said@gmail.com', 'i.larsson@gmail.com', 'f.degass@gmail.com', 'i.rossi@gmail.com', 'jpkm89@hotmail.com', 'orwellt@yahoo.com'],
    'attended': [True, True, False, True, True, False, True, True, False, False],
    'class': ['B2', 'A2', np.NaN, 'B1','C2', np.NaN, 'B2', 'B1', np.NaN, np.NaN],
    'avg_score': [3.2, 3.3, np.NaN, 2.9, 'three', np.NaN, 3.9, 3.5, np.NaN, np.NaN]})

# alternatively you can also define it row-wise
data = [
    ['user6', 'Ingrid', 'Larsson', 19, 'i.larsson@gmail.com', False, np.NaN, np.NaN],
    ['user7', 'Francois', 'Degass', 22, 'f.degass@gmail.com', True, 'B2', 3.9],
    ['user8', 'Ilaria', 'Rossi', 229, 'i.rossi@gmail.com', True, 'B1', 3.5]
]
columns = ['index', 'name', 'surname', 'age', 'email', 'attended', 'class', 'avg_score']
df1 = pd.DataFrame(data=data, columns=columns)

df2 = pd.DataFrame({'sex': [1, 1, 1, 0, 1, 0, 1, 0]})

# you can also specify index from a list or from an existing dataframe
df1 =  pd.DataFrame(data, columns=columns, index=range(len(data)))

#### write df

In [ ]:
# csv
# write pandas df to csv
df.to_csv('output.csv', index=False)
# write pandas df containing special characters to csv
df.to_csv(output_path, index=False, encoding='utf-8-sig')

# write df to json or other weird formats
with open(output_path, 'wb') as file:
    dump(df, file)

In [ ]:
# excel
# write pandas df to new excel
df.to_excel('output.xlsx', sheet_name='sheet', index=False)
# write pandas df containing special characters to excel
df.to_excel(output_path, engine='xlsxwriter', index=False)

# write multiple pandas df to same new excel
with pd.ExcelWriter('output.xlsx') as writer:
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    model_dependencies.to_excel(writer, sheet_name='model_dependencies', index=False)
    writer.save()
    
# write pandas df to existing excel
from openpyxl import load_workbook
book = load_workbook('EmailAssistsCfr.xlsx')
writer = pd.ExcelWriter('EmailAssistsCfr.xlsx')
writer.book = book
df.to_excel(writer, sheet_name='EmailAssistedLeads', index=False, header=True)
writer.save()
writer.close()

# write excel containing unicode
os.cmd('pip install xlsxwriter')
df.to_excel('test.xlsx', engine='xlsxwriter')

#### read df

In [ ]:
# read csv
df = pd.read_csv('Data/Titanic/train.csv')
# read encoded csv file
df = pd.read_csv('anl_CRY_FullClientList_clientContactInfo_V2.csv', sep=';', encoding='ISO-8859-1')

# read excel
df = pd.read_excel(path, usecols=['id', 'name', 'cellphone'])

# read multiple sheets from excel
dfs = []
for sheet in sheets:
    df = pd.read_excel(file_name, sheet_name=sheet, usecols=[cols_to_read])
    dfs.append(df)
dfs = pd.concat(dfs).drop_duplicates()

# read multiple files
dfs = []
for file in files:
    df = pd.read_csv(file, usecols=columns_list)
    dfs.append(df)
dfs = pd.concat(dfs)

<a name="handling"></a>
### simple df handling

[Return to Contents](#contents)

In [ ]:
# --------------------- DESCRIPTIVE STATISTICS
df.dtypes                                                        # check columns types
df.head(n)                                                       # select first n rows
df.tail(n)                                                       # select last n rows
df.sample(n)                                                     # select random subset of n rows
df.describe()                                                    # generate descriptive statistics about df
df.columns                                                       # show list of columns
df.columns[df.dtypes=='object']                                  # find categoric / numeric columns
pd.factorize(df[col])                                            # map categorical values to integers, extract unique values

# FOCUS: describe
df.describe([.01,.05,.25,.5,.75,.9,.95,.99])
df.describe(include=['object'])
df.describe(include=['category'])
df.describe(datetime_is_numeric=True)

In [ ]:
# --------------------- RESHAPE DATAFRAME
df.set_index(col)                                                # set col as index
df.reset_index()                                                 # reset index to row nr
df.index.name = None                                             # remove index name
df.rename_axis(None)                                             # remove axis name
df.transpose()                                                   # transpose df
df.reindex(idx) / df.reindex(idx, axis=1)                        # conform df to new index with optional filling logic (can be done on row indexes or on columns)
pd.melt(df)                                                      # gather columns into rows
df.rename({col:new_col}, axis=1)                                 # rename columns
df.drop([col1, col2], axis=1, errors='ignore')                   # drop list of columns

# FOCUS: index
# to change the index and then restore the index level you have to remove index name after resetting it
df = df.set_index(col)
df.index.name = None
# creating a multi-index
# below example shows how to create a column multi-index through a dictionary d, however the same can be done with rows
df.columns = pd.MultiIndex.from_tuples([(d[k], k) for k in df.columns])
# to concatenate multi-indexes into one
df.columns.map('_'.join).str.lower()

In [ ]:
# --------------------- SORT DATAFRAME
df.sort_index()                                                  # sort by index
df.sort_values(by=[col1, col2], ascending=False)                 # sort by given columns
df.sort_values([(lvl1, lvl2)])                                   # sort by multi-index

# sort by custom list
sort_list = []
df.column = df.column.astype('category')
df.column.cat.set_categories(sort_list, inplace=True)
df = df.sort_values(['column'])

# FOCUS: sorting with multi-index
# here you can specify whatever number of index levels and ordering
# let's first create a multi-index and then sort it
df.columns = pd.MultiIndex.from_tuples([(d[k], k) for k in df.columns])
df.sort_index(axis=1, level=[0, 1], ascending=[True, False], inplace=True)

# FOCUS: change data type
df[col] = df[col].astype('category')
df[col] = df[col].astype('int')
df[col] = df[col].astype('float')
# to convert a column containing nans to int use instead Int64
df[col] = df[col].astype('Int64')

In [ ]:
# --------------------- SLICE DATAFRAME
# the simplest way to slice a pandas dataframe is using a boolean series
# the usage does not depend on the index axis (whether is it the default index or a user defined index)
df[df['age'] > 30]
df[df['age'].isin([20, 25, 30, 35])]
df[df['age'].between(18, 24)]

# label-based slicing
# loc is label-based meaning that rows and columns have to be specified by their names
df.loc[index_val]                                                # scalar indexing
df.loc[index_vals]                                               # list indexing
df.loc[bool_list]                                                # list of booleans indexing: it has to be the same length of the dataframe!
df.loc[index_val, 'age']                                         # indexing both axis
df.loc[df['age'] > 30, 'email']                                  # series indexing

# index-based slicing
# iloc is integer index-based meaning that rows and columns have to be specified by their integer indexes
# this is useful when you know the index you are interested in or when you are looping throung the dataframe
df.iloc[1]                                                       # scalar integer indexing: this will return a Series obj
df.iloc[[1]]                                                     # list of integers indexing: this will return a DataFrame obj
df.iloc[bool_list]                                               # list of booleans indexing: it has to be the same length of the dataframe!
df.iloc[1, 2]                                                    # indexing both axis
# NB iloc does not work with Series indexing, to solve this you have to convert the Series to list
df.iloc[(df['age'] > 20).to_list()]

# other slicing techniquest
# slicing based on map functions
df[df['list'].map(len) > 1]
# select rows based on regex
df.filter(regex='e$', axis=1)
# select n largest / smallest rows
df.nlargest(5, 'age')
df.nsmallest(5, 'age')

# FOCUS: slicing with multi-index
df.columns = pd.MultiIndex.from_tuples([(d[k], k) for k in df.columns])
# the first method allows to slice rows
df[df[('idx_lvl1','idx_lvl2')].isin([2805803003,2806501014])]
# otherwise you can take advantage of pandas loc and indexSlice methods to slice rows and columns
idx = pd.IndexSlice
df.loc[:, idx[['a_sección','renta'],:]]
# WIP
# selezionare una colonna da un indice e tutte le colonne da un altro indice

In [ ]:
# --------------------- MISSING VALUES
df.drop_duplicates()                                             # drop duplicated rows

# counting missing values
df.isna().sum()
df.isnull().sum().sort_values(ascending=False)

# visualising missing values
df[df['class'].isna()].head()
df[df['class'].isnull()]
df[df.isna()].head()

# filling missing values in a column
df['avg_score'] = df['avg_score'].fillna(0)

# filling missing values in all dataframe
# fill with user defined value like empty string
df = df.fillna('')
# to fill with median
replace_dict = df.median().to_dict()
df= df.fillna(replace_dict)
# to fill with values from another column
df['name'] = df['name'].fillna(df['surname'])

# drop missing values
df = df.dropna()

In [ ]:
# --------------------- DUPLICATED VALUES
# get first occurrence of duplicated values from column
df[df.duplicated(subset='name')]
df[df.duplicated(subset=['name', 'surname'])]

# get all rows that have duplicates in column
df[df['surname'].isin(df[df.duplicated(subset='surname')]['surname'].to_list())]

In [ ]:
# create date from string columns
df['SendDate'] = df['SendDate'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
df['SendDate'] = df['SendDate'].dt.date

# fill date columns
df['SendDate'] = df['SendDate'].fillna(dt.date(1900, 1, 1))

# extract info from date
df['Year'] = df['SendDate'].apply(lambda x: x.strftime('%Y'))
df['Month'] = df['SendDate'].apply(lambda x: x.strftime('%B'))

<a name="reshape"></a>
### reshape df

[Return to Contents](#contents)

In [ ]:
df.head()

In [ ]:
# spread rows into columns
# here index denotes the column that will be kept as index (MUST be single index)
# columns denotes the columns that will be used as columns
# values will be the values placed inside the table
df_pivot = df.pivot(index=['name','surname'], columns='class', values='avg_score')

# after pivoting you may want to concatenate indexes in order to have them in one row
# this can be done with the following
# df_pivot = df_pivot.columns.map('_'.join).str.lower()

df_pivot

In [ ]:
# the "inverse" operation of a pivot is melt statement
#  - id_vars: columns to be used as row identifier, these columns not be moved
#  - value_vars: columns to unpivot, these columns' names will be placed in a column named 'variable'
# the values in value_vars will be then used to populate a single column
df_metl = pd.melt(df_pivot.reset_index(), id_vars=['name','surname'], value_vars=['A2', 'B1', 'B2', 'C2'])
df_metl.head()

<a name="slice"></a>
### slice df

[Return to Contents](#contents)

In [ ]:
# you can slice a dataframe with custom methods through apply method
# note that apply requires the column to be a iterable (list, set, tuple), if that is not the case make sure to convert it to iterale before slicing
df[df.mq_range_list.apply(lambda x: bool(set(x) & set(l)))].head()

#### loc

Pandas `loc[]` allows to access a DataFrame's group of rows and columns by label(s) or a boolean array.

In [ ]:
df.loc[len(df)]                                                  # access DataFrame's last row
df.loc[len(df), 'name']                                          # access a specific value
df.loc[df['age'] < 50, ['name','email']]                         # select subset of rows and columns

# loc can be used to modifiy data in a subset of rows
df.loc[df['f_outlier'] == 1, 'fatturato'] = np.NaN

# select a subset of dataframe based on indexes
df = y.loc[X.index]

#### iloc

Pandas `iloc[]` property is a purely integer-location based indexing for selection by position.

In [ ]:
df.iloc[0]                                                       # access DataFrame's first row
df.loc[len(df)]                                                  # access DataFrame's last row
df.iloc[0]['name']                                               # access a specific value

# select a subset of dataframe based on indexes
df = y.iloc[X.index]

### binning

[Return to Contents](#contents)

In [ ]:
# static binning
# function cut splits a serie in bins according to pre-defined (static) bins
bins = [0, 2, 3.1, 5]
df['comment'] = pd.cut(df['avg_score'], bins)

# quantile binning
# quantile-cut applies a number q of quantile splits to the serie
df['quantile_comment'] = pd.qcut(df['avg_score'], q=nr_bins, labels=comment)
# qcut actually returns the intervals (pandas.Interval) as you can see below
df['age_qbins'] = pd.qcut(df.age, q=3).values
# use methods such as left, right to access each interval's boundary
df['age_boundary'] = df['age_qbins'].apply(lambda x: x.right)

<a name="rowwise"></a>
### row-wise operations

[Return to Contents](#contents)

In [ ]:
# create column based on regex
# extract: returns first match, if it does not match returns nan
df['test'] = df['hdfs'].str.extract(r'name\=(.+)')
# findall: returns list of matches, if it does not match returns empty list
df['test'] = df['hdfs'].str.findall(r'name\=(.+)')

#### replace

In [ ]:
# replace values statically
# use alternatively
df['avg_score'].replace({'three': 3.}, inplace=True)
df['avg_score'] = df['avg_score'].replace({0: 0.})

# replace multiple values with the same one
df['age'] = df['age'].replace([14, 15, 16, 17], 18)

<a name="map"></a>
#### map

Map is a Series operation that maps values of Series according to input correspondence

**Note** that `map()` method is actually not a row-wise operation

In [ ]:
df['Title'] = df['Title'].map({'Miss':'Mrs', 'Ms':'Mrs'})

<a name="apply"></a>
#### apply

In [ ]:
# apply simple row-wise operation to column
df[new_column] = df[old_column].apply(lambda x: ' '.join(x.split('|')))

# you can also apply row-wise operations that operate over multiple columns by applying directly to df
# in order to do that you have to specify axis=1 to apply function
df[new_column] = df.apply(lambda x: x.first_column if x.second_column else 'NA', axis = 1)

# you can also define custom logics by writing your own functions
def getTech(legend):
        tech = None
        tech_initial = legend[1]
        if(tech_initial == 'L') :
            tech = 'LTE'
        elif(tech_initial == 'U') :
            tech = 'UMTS'
        elif(tech_initial == 'G') :
            tech = 'GSM'
        return tech
df['tech'] = df.apply(lambda x: getTech(x['legend']), axis=1)

In [ ]:
# you can create multiple columns at once with apply
def get_cell(legend):
    legend_inital = legend[1]
    if legend_inital in ['D', 'G', 'M', 'N']:
        tech = 'GSM'
        gen = '2G'
        zci = legend[0] + legend[2:7] + legend[-1]
    elif legend_inital in ['H', 'U', 'V', 'X']:
        tech = 'UMTS'
        gen = '3G'
        zci = legend[0] + legend[2:7] + legend[-1]
    elif legend_inital in ['L']:
        tech = 'LTE'
        gen = '4G'
        zci = legend[6:11] + '0' + legend[14:16]
    else:
        tech = 'N/A'
        gen = 'N/A'
        zci = 'N/A'
    return pd.Series([tech, gen, zci])

# mtd 1: return a pd.Series
coverage[['TECH', 'GEN', 'ZCI']] = coverage['LEGEND'].apply(get_cell)

# mtd 2: use zip
scraping_out['hours_info'], scraping_out['totale_ore'], scraping_out['orario_continuato'], scraping_out['aperto_24h'] = zip(*scraping_out['hours'].apply(extract_hour_info))

# split column into multiple columns
df[['codice11', 'codice11_dest']] = pd.read_table(io.StringIO(df['Name'].to_csv(None, index=False, header=False)), sep=' - ', dtype=str, header=None)

<a name="aggregate"></a>
### aggregate operations

[Return to Contents](#contents)

<a name="groupby"></a>
#### groupby

Generally, a `groupby` function consists of three steps:
- split
- apply
- combine

The split step breaks up the dataframe into subset dataframes based on the specified keys.  
Then, apply step applies functions to those subset dataframes.  
Last, combine step concatenates those results into an output array.

**Note** that groupby method returns a pandas DataFrameGroupBy object with the group column(s) as index.
Remember to use `reset_index` after groupby if you want to have group columns on the same level as the other columns

In [ ]:
# simple groupby
df.groupby(['Pclass']).mean()

# grouping on multiple columns
df.groupby(['Sex', 'Pclass']).mean()

# resetting index
df.groupby(['Pclass']).mean().reset_index()
df.groupby(['Pclass'], as_index=False).mean()

# imputing missing values
df['Deck'] = df.groupby(by=['Ticket'], sort=False)['Deck'].apply(lambda x: x.ffill().bfill())


# handling missing values (pandas 1.3.3)
# df_missing = df.copy()
# df_missing.iloc[80:100, 0] = np.nan
# df_missing.groupby(by='Pclass', dropna=False).mean()

<a name="aggr"></a>
#### aggregation

Aggregation means that an aggregate DataFrame will be retreived with a number of rows matching the number of classes in the column(s) on which the `goupby` is performed.  
Basically aggregation allows to apply multiple functions to a subset of columns and can be done in pandas with the `aggr` method.

The aggregation logical steps are the following.  
First, it splits the full dataframe into sub DataFrames based on the grouping column(s).  
Then it applies aggregation function(s) to the specified column(s).  
Last, it combines the results into series that can be converted to DataFrame through `reset_index`.  

In [ ]:
# simple operations
# NOTE that if you specify a column, like Age in the following, you get aggregations only on that specific one
# you can get aggregations on all columns if you avoid specifying one
df.groupby('Sex').Age.max()

# agg method
df.groupby('Sex').Age.agg(['max', 'min', 'count', 'median', 'mean'])
# to aggregate multiple columns with custom operations
df.groupby('Sex').agg({'Age': ['min', 'max'], 'Fare': 'sum'})
# to rename columns instead use the following method
df.groupby('Sex').Age.agg(sex_max=('max'), sex_min=('min'))
df.groupby('Sex').agg(age_max=('Age','max'), age_min=('Age','min'), fare_sum=('Fare','sum'))

# nice functions
# numpy ptp (peak to peak) retrieves the range of values (maximum - minimum) along an axis
df.groupby('Sex').agg(age_delta=('Age',np.ptp), fare_delta=('Fare',np.ptp))

# to use custom functions
def categorize(x):
    return True if x.mean() > 29 else False
df.groupby('Sex').Age.agg(['max', 'mean', categorize])

# or with lambda expression
df.groupby('Sex').Age.agg(['max', 'mean', lambda x: True if x.mean() > 50 else False])

# alternatively use APPLY method when performing aggregation on whole column and NOT on single elements (see below)
df.groupby('Pclass').mean().head()
df.groupby('Pclass').apply(lambda x: x.mean()).head()

<a name="transf"></a>
#### transformation

Transformation returns a list-indexed object with the same number of rows as the input DataFrame and can be done in pandas with the `transform` method.

Compared with aggregation, transformation takes an additional step called *Broadcasting*.
Broadcasting consists of returning the results from sub DataFrames to the original full DataFrame.
Thus transform will always return a series with the same length to the original full DataFrame.

In [ ]:
df.groupby('Pclass').transform(lambda x: x.mean()).head()

# to use custom functions
standardization = lambda x: (x - x.mean()) / x.std()
df.groupby('Sex').Age.transform(standardization)

# NOTE that when the apply function is applied to single element the it behaves exactly as the transform function
df.groupby('Sex').Age.apply(standardization)

<a name="filter"></a>
#### filter

Finally, filtering means discarding some observations according to a group-wise computation that evaluates True or False.
Filtering in pandas is done by the `filter` method.

Compared with `transform`, `filter` takes another additional step.  
After getting results from those sub DataFrames, it applies a filter condition to those results and then it broadcasts the result to the matching conditions in the original DataFrame.  
Basically it is simply reducing the original DataFrame's dimension according to a grouping condition.

In [ ]:
# simple filter
df.groupby('Cabin').filter(lambda x: len(x) >= 4)

<a name="combine"></a>
### combining operations

[Return to Contents](#contents)

Pandas supports different methods to combine dataframes such as `join`, `merge` and `concat`.

Both `join` and `merge` can be used to combines two dataframes but they are slightly different: `join` method combines two dataframes on the basis of their indexes whereas `merge` method is more versatile and allows user to specify columns beside the index to join on for both dataframes.  
`concat` method on the other side allows to concatenate a list of pandas objects along a particular axis with optional set logic along the other axes. 

**Remember** that both `join` and `merge` reset the output dataframe index so in order to preserve the original index you have to reset it before the join or merge operation


**References**
- https://pandas.pydata.org/docs/user_guide/merging.html

<a name="combine"></a>
#### join

Among the three operations, `join` is the one that allows the lowest level of control.  
It is usually used to join data based on their indexes and it will combine all the columns from the two tables, with the common columns renamed with the defined `lsuffix` and `rsuffix`. 

In [ ]:
join_df = df1.join(df2)
# outer join

# anti join
outer_join = left_df.merge(right_df, how='outer', indicator=True)
# then the column _merge will be populated with the following values: both, left_only, right_only
# hence you can derive the anti join with the following line
anti_join = outer_join[~(outer_join._merge == 'both')]

<a name="merge"></a>
#### merge

Similarly to `join`, `merge` also combines all the columns from the two tables, with the common columns renamed with the defined suffixes however, it provides three ways of flexible control over row-wise alignment.  
The first way is to use `on=COLUMN_NAME`, here the given column must be the common column in both tables.  
The second way is to use `left_on=LEFT_COLUMN_NAME` and `right_on=RIGHT_COLUMN_NAME` , and it allows to align the two tables using two different columns.  
The third way is to use `left_index=True` and `right_index=True`, and the two tables are aligned based on their index.

**Note** that `merge` is more powerful that `join` resulting in some circumstances `join` not being able to actually join tables (and raining data format error) whereas `merge` can.  
Prefer `merge` over `join` if you are not joining on DataFrames' indexes.

In [ ]:
# dataframe method
left_df.merge(right_df)

# function
pd.merge(left_df, right_df)

<a name="combine"></a>
#### concat

Different from `join` and `merge`, which by default operate on columns, `concat` can define whether to operate on columns or rows.  
By default it operates on rows `axis=0` but user can specify `axis=1` to operate on columns.

`concat` is useful when generating/reading dataframes in loop with the same schema.  
These dataframes can be stored in a list during the loop and finally be concatenated to get a unique dataframe storing all data.

In [ ]:
# append rows to a dataframe
concat_df = pd.concat([df, df1])
# use flag ignore_index=True to not use the index values along the concatenation axis
concat_df = pd.concat([df, df1], ignore_index=True)

# append columns to dataframe
concat_df = pd.concat([df, df2], axis=1)

<a name="encoding"></a>
### encoding

[Return to Contents](#contents)

In [ ]:
# one-hot encoding of dummy columns
pd.get_dummies(df, columns=list_of_columns, prefix=prefix_to_encoded_columns)

<a name="complex"></a>
### complex operations

[Return to Contents](#contents)

In [ ]:
# aggregate a list of tuples and keep tuple with the maximum second element
# NB in order to aggregate, list have to be sorted by the element on which you will be aggregating
tuples_list = tag_arr_ = [('piero', 1), ('franco', 0.5), ('lillo', 3), ('dome', 0.1), ('franco', 2)]
tuples_list.sort(key=lambda tup: tup[1])
[tup for tup in [max(v, key=itemgetter(0)) for k, v in groupby(tuples_list, itemgetter(1))]

In [ ]:
# TODO: move this functions to correct locations depending on usage

# explode lists
df.explode('column', ignore_index=True)

# transpose
df.T

# automatically try to infer best data type for object column
df.convert_dtypes()

# pandas options
# you can surf pandas options with
dir(pd.options)
dir(pd.options.display)
# and then changing configuration as
pd.options.display.max_columns = None
pd.options.display.precision = 5

# pandas styler
# you can change style to dataframes
df.style.highlight_max(color='darkred')
df.style.background_gradient(subset=['mean', '50%'], cmap='Reds')

# pipe together custom functions
# here drop_duplicates and remove_outliers are assumed to be custom functions with their own input and outputs
df = df.pipe(drop_duplicates).pipe(remove_outliers, ['price', 'carat', 'depth'])

---
<a name="gpd"></a>
# Geopandas

[Return to Contents](#contents)

Per disegnare bounding boxes: https://arthur-e.github.io/Wicket/sandbox-gmaps3.html

In [ ]:
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point, Polygon

In [ ]:
# read csv
maps = pd.read_csv('../../Dati/vf_maps.csv')
maps['geometry'] = maps['geometry'].apply(wkt.loads)
maps_gpd = gpd.GeoDataFrame(maps, geometry='geometry', crs={'init': 'epsg:4326', 'no_defs': True})

# read shapefile
maps = gpd.read_file('../../Shapefiles/atoll_202012/atoll_2g_coveragemaps_20201231.shp').to_crs(epsg=4326)

In [ ]:
# POINTS
# create points from scratch
point = Point(6.681902735071699, 46.02450054611408)

# create points from pandas columns
df['geometry'] = df.apply(lambda x: Point(x.longitude, x.latitude), axis=1)

# POLYGONS
# create polygon from scratch
boundingbox_polygon = Polygon([p1, p2, p3, p4])

# create polygons from shapely points
# when you have points from a single polygon
df['polygon'] = Polygon([(p.x, p.y)  for p in  df.point])
# when you have points from multiple polygons (with different id)
df = df.groupby('id')['geometry'].apply(lambda points: Polygon([(p.x, p.y)  for p in points.tolist()])).reset_index()

# GPD DF
# create gpd dataframe from scratch
df_gpd = gpd.GeoDataFrame(gpd.GeoSeries(boundingbox_polygon), columns=['geometry'])
# create gpd dataframe from pandas
df_gpd = gpd.GeoDataFrame(df, geometry='polygon', crs={'init': 'epsg:4326', 'no_defs': True})

In [ ]:
# union geopandas dataframe
# pd.concat trasforms to pandas df
coverage = coverage2g.append(coverage3g).append(coverage4g)

# select / drop columns
# using df = df[[...]] will convert dataframe to pandas instead of geopandas
df_gpd = df_gpd.drop(columns=['index_left', 'L_ID_P', 'A_ID_P'])

In [ ]:
# centroids
df['centroid'] = df.polygon.centroid

# convex hull
df['convex_hull'] = df.polygon.convex_hull
df['hull'] = df['geometry'].apply(lambda x: x.convex_hull)

# aggregate geometries
# dissolve will group obs by given column and then aggregate geometries
df = df.dissolve(by='id').reset_index()

# apply buffer to geometry
def buffer(row):
    return row.geometry.buffer(0.01, resolution=4, cap_style=3, join_style=2)
df['geometry'] = df.apply(buffer, axis=1)

In [ ]:
# spatial join
sjoin = gpd.sjoin(maps_gpd, coverage_gpd, how='inner', op='contains')

# overlay
overlay = gpd.overlay(maps_gpd, coverage_gpd, how='intersection')
# sort by intersection area so that largest intersection area is last
overlay.sort_values(by='geometry', inplace=True, key=lambda col: np.array([x.area for x in col]))
# keep largest intersection area for each cell
overlay.drop_duplicates(subset='LEGEND', keep='last', inplace=True)

In [ ]:
# write csv containing geometry
df.to_csv(filename, index=False, encoding='utf-8-sig')

# write csv without geometry
df.to_csv(os.path.join(output_path, 'VFA_IN_GCP_CONFIG_VF_MAPS_vf_maps_{}.csv'.format(curr_date)), index=False)